In [0]:
# !mkdir ./data
#!wget -N https://www.dropbox.com/s/vcgh3wfbazvj0ll/parameters.zip?dl=0 -O ./data/parameters.zip
# !unzip ./data/parameters -d ./data

import torch
import torch.nn as nn
import torchvision.transforms as transforms

import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

from LIP_Model import PoseRefineNet, ParsingRefineNet, PoseNet, ParsingNet
from Model import BackboneResNet

In [0]:
# !wget -N https://www.dropbox.com/s/wfemdhn62zpxdud/backbone_parameter.zip?dl=0 -O ./backbone.zip
# !unzip ./backbone.zip -d ./

In [3]:
device = "cpu"

parsing_net = ParsingNet(20).to(device)
pose_net = PoseNet().to(device)
pose_refine = PoseRefineNet(20).to(device)
parsing_refine = ParsingRefineNet(20).to(device)
backbone = BackboneResNet().to(device)

# backbone.load_state_dict(torch.load("data/backbone_parameter.pt"))
backbone.load_state_dict(torch.load("data/backbone_parameter.pt"))
parsing_net.load_state_dict(torch.load("data/parsing_net_parameter.pt"))
parsing_refine.load_state_dict(torch.load("data/parsing_refine_net_parameter.pt"))
pose_net.load_state_dict(torch.load("data/pose_net_parameter.pt"))
pose_refine.load_state_dict(torch.load("data/pose_refine_net_parameter.pt"))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [0]:
IMG_MEAN = np.array((104.00698793,116.66876762,122.67891434), dtype=np.float32)

class myTensor(object) :
    def __call__(self, pic) :
        if isinstance(pic, np.ndarray) :
            pass
        elif Image.isImageType(pic) :
            pic = np.array(pic)
            pic_r, pic_g, pic_b = np.split(pic, 3, axis=2)
            pic = np.concatenate([pic_b, pic_g, pic_r], axis=2)
            pic = pic - IMG_MEAN
            pic = torch.from_numpy(pic).permute(2,0,1)
            pic = pic.unsqueeze(0)
            return pic
        else : # error
            pass

In [6]:
# RGB를 BGR로도 나중에 해보자

# img = Image.open("2583_462340.jpg")
img = Image.open("100034_483681.jpg")
# img = Image.open("100142_449784.jpg")

# img_resize = img.resize((384, 384))

# m = torch.nn.UpsamplingBilinear2d(size=(384, 384))
# img_100 = m(img_100)

transform_100 = transforms.Compose([myTensor(), torch.nn.UpsamplingBilinear2d(size=(384, 384))])
transform_075 = transforms.Compose([myTensor(), torch.nn.UpsamplingBilinear2d(size=(int(384 * 0.75), int(384 * 0.75)))])
transform_125 = transforms.Compose([myTensor(), torch.nn.UpsamplingBilinear2d(size=(int(384 * 1.25), int(384 * 1.25)))])

img_100 = transform_100(img)
img_075 = transform_075(img)
img_125 = transform_125(img)

img_100_reverse = torch.flip(img_100, dims=[3])
img_075_reverse = torch.flip(img_075, dims=[3])
img_125_reverse = torch.flip(img_125, dims=[3])

input_img_100 = torch.cat((img_100,img_100_reverse), dim=0)
input_img_075 = torch.cat((img_075,img_075_reverse), dim=0)
input_img_125 = torch.cat((img_125,img_125_reverse), dim=0)


print(input_img_100.size())
print(input_img_075.size())
print(input_img_125.size())
permuted = input_img_100.permute(0, 2, 3, 1)
torch.set_printoptions(profile="full")
print(permuted[0][0])
torch.set_printoptions(profile="default")

torch.Size([2, 3, 384, 384])
torch.Size([2, 3, 288, 288])
torch.Size([2, 3, 480, 480])
tensor([[ 150.9930,   95.3312,   98.3211],
        [ 150.0217,   94.3600,   97.3498],
        [ 149.0504,   93.3887,   96.3785],
        [  92.7920,   37.1302,   40.1200],
        [  33.0583,  -22.6035,  -19.6136],
        [   5.0113,  -49.7941,  -47.2324],
        [ -18.7851,  -72.6192,  -70.5432],
        [  -9.4247,  -61.0891,  -59.8982],
        [   7.0870,  -42.1492,  -41.9296],
        [  12.4760,  -35.4442,  -35.4544],
        [  14.4186,  -32.5304,  -32.5405],
        [  19.0975,  -26.8803,  -27.2324],
        [  24.9251,  -20.0813,  -20.9191],
        [  10.3873,  -32.7079,  -34.3447],
        [ -15.3517,  -56.0187,  -58.6267],
        [ -14.9078,  -53.1466,  -56.4413],
        [   4.0322,  -31.7784,  -35.5588],
        [  48.3037,   15.6889,   11.4229],
        [ 115.3220,   86.5923,   81.8406],
        [ 150.9930,  127.0571,  122.0469],
        [ 150.9930,  132.8848,  127.8746],
        [ 

## input_img_100

In [7]:
feature_map = backbone(input_img_100)

print(feature_map.size())

# parsing_out1, parsing_fea1 = parsing_net(feature_map)

torch.Size([2, 1024, 48, 48])


In [8]:
print(feature_map.size())
feature_p = feature_map.permute(0, 2, 3, 1)
print(feature_p.size())
torch.set_printoptions(profile="full")
print(feature_p[0][0][0])
#for i in feature_p[0][0]:
#  print(i)
torch.set_printoptions(profile="default")


torch.Size([2, 1024, 48, 48])
torch.Size([2, 48, 48, 1024])
tensor([1.2617e+00, 3.5344e+00, 9.7394e-01, 1.2390e+00, 2.9408e-01, 0.0000e+00,
        4.0173e+00, 1.3207e+00, 2.2931e+00, 6.2118e-01, 5.8451e-01, 1.2262e+00,
        1.3694e+00, 8.9383e-01, 2.8856e+00, 1.6554e+00, 1.2569e+00, 2.4386e+00,
        1.4123e+00, 7.6198e-01, 1.8715e+00, 2.5839e+00, 1.9420e+00, 2.2733e-01,
        1.8973e+00, 1.3697e+00, 1.2691e+00, 1.5060e+00, 2.0303e+00, 1.7273e+00,
        3.0673e+00, 1.0911e+00, 1.0314e+00, 1.5271e-01, 2.4294e-01, 1.2705e+00,
        9.9403e-01, 1.2574e+00, 1.5481e+00, 0.0000e+00, 1.1732e+00, 2.2564e+00,
        1.8763e+00, 1.8872e+00, 1.3671e+00, 5.9516e-02, 0.0000e+00, 2.9695e+00,
        1.4051e+00, 3.5770e+00, 3.7506e+00, 1.5488e+00, 0.0000e+00, 2.3036e+00,
        0.0000e+00, 2.1381e+00, 3.3926e+00, 3.7715e+00, 0.0000e+00, 1.8501e+00,
        1.6723e+00, 1.4792e-01, 1.2834e-01, 5.7488e-01, 2.2028e+00, 1.3749e+00,
        1.7024e+00, 1.9128e+00, 2.7590e+00, 6.2002e-01, 2.07

In [0]:
pose_map1, pose_context1 = pose_net(feature_map)
pose_map2, pose_context2 = pose_refine(pose_map1, parsing_out1, pose_context1)
parsing_out2, parsing_fea2 = parsing_refine(pose_map1, parsing_out1, parsing_fea1)
pose_map3_100, pose_context3_100 = pose_refine(pose_map2, parsing_out2, pose_context2)

In [0]:
print(pose_map1.size())

head_output, tail_output = torch.unbind(pose_map1, dim=0)
tail_list = torch.unbind(tail_output, dim=0)

tail_list_rev = [None] * 16
tail_list_rev[0] = tail_list[5]
tail_list_rev[1] = tail_list[4]
tail_list_rev[2] = tail_list[3]
tail_list_rev[3] = tail_list[2]
tail_list_rev[4] = tail_list[1]
tail_list_rev[5] = tail_list[0]
tail_list_rev[10] = tail_list[15]
tail_list_rev[11] = tail_list[14]
tail_list_rev[12] = tail_list[13]
tail_list_rev[13] = tail_list[12]
tail_list_rev[14] = tail_list[11]
tail_list_rev[15] = tail_list[10]
tail_list_rev[6] = tail_list[6]
tail_list_rev[7] = tail_list[7]
tail_list_rev[8] = tail_list[8]
tail_list_rev[9] = tail_list[9]

tail_output_rev = torch.stack(tail_list_rev, dim=0)
tail_output_rev = torch.flip(tail_output_rev, dims=[2])

output_all = torch.mean(torch.stack((head_output, tail_output_rev)), dim=0, keepdim=True)

img = img.resize((384, 384))
plt.imshow(img)

for c in range(output_all.shape[1]) :
    channel = output_all[0][c]
    x, y = np.unravel_index(channel.cpu().argmax(), channel.shape)
    plt.scatter(y, x, linewidths=3, c="red")
    print(y, x)

## input_img_075

In [0]:
feature_map = backbone(input_img_075)

parsing_out1, parsing_fea1 = parsing_net(feature_map)

In [0]:
print(feature_map.size())

In [0]:
pose_map1, pose_context1 = pose_net(feature_map)
pose_map2, pose_context2 = pose_refine(pose_map1, parsing_out1, pose_context1)
parsing_out2, parsing_fea2 = parsing_refine(pose_map1, parsing_out1, parsing_fea1)
pose_map3_075, pose_context3_075 = pose_refine(pose_map2, parsing_out2, pose_context2)

## input_img_125

In [0]:
feature_map = backbone(input_img_125)

parsing_out1, parsing_fea1 = parsing_net(feature_map)

In [0]:
print(feature_map.size())

In [0]:
pose_map1, pose_context1 = pose_net(feature_map)
pose_map2, pose_context2 = pose_refine(pose_map1, parsing_out1, pose_context1)
parsing_out2, parsing_fea2 = parsing_refine(pose_map1, parsing_out1, parsing_fea1)
pose_map3_125, pose_context3_125 = pose_refine(pose_map2, parsing_out2, pose_context2)

## 결과

In [0]:
print(pose_map3_100.size())
print(pose_map3_075.size())
print(pose_map3_125.size())

In [0]:
pose_out = torch.mean(torch.stack((nn.functional.interpolate(pose_map3_100, size=(384, 384)),
                                     nn.functional.interpolate(pose_map3_075, size=(384, 384)),
                                     nn.functional.interpolate(pose_map3_125, size=(384, 384)))), dim=0)

head_output, tail_output = torch.unbind(pose_out, dim=0)
tail_list = torch.unbind(tail_output, dim=0)

In [0]:
tail_list_rev = [None] * 16
tail_list_rev[0] = tail_list[5]
tail_list_rev[1] = tail_list[4]
tail_list_rev[2] = tail_list[3]
tail_list_rev[3] = tail_list[2]
tail_list_rev[4] = tail_list[1]
tail_list_rev[5] = tail_list[0]
tail_list_rev[10] = tail_list[15]
tail_list_rev[11] = tail_list[14]
tail_list_rev[12] = tail_list[13]
tail_list_rev[13] = tail_list[12]
tail_list_rev[14] = tail_list[11]
tail_list_rev[15] = tail_list[10]
tail_list_rev[6] = tail_list[6]
tail_list_rev[7] = tail_list[7]
tail_list_rev[8] = tail_list[8]
tail_list_rev[9] = tail_list[9]

In [0]:
tail_output_rev = torch.stack(tail_list_rev, dim=0)
tail_output_rev = torch.flip(tail_output_rev, dims=[2])

output_all = torch.mean(torch.stack((head_output, tail_output_rev)), dim=0, keepdim=True)

In [0]:
img = img.resize((384, 384))
plt.imshow(img)

for c in range(output_all.shape[1]) :
    channel = output_all[0][c]
    x, y = np.unravel_index(channel.cpu().argmax(), channel.shape)
    plt.scatter(y, x, linewidths=3, c="red")
    print(y, x)

In [0]:
plt.imshow(output_all[0][0].cpu().detach().numpy())